<a href="https://colab.research.google.com/github/VictoKu1/Anomaly_Detection/blob/master/Task1_Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Anomaly Detection**

---

---

##### ***Authors:*** *Moriya Bitton & Victor Kushnir*

---



---


##### **[Github Link](https://github.com/VictoKu1/Anomaly_Detection)**

## **Part 0 - Imports** 

---



---



In [63]:
import io
import numpy as np
import pandas as pd
import seaborn as sns
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix, f1_score

## **Part 1 - Data** 

---



---



In [ ]:
data_to_load = files.upload()
target_to_load = files.upload()

Saving conn_attack.csv to conn_attack (2).csv


In [ ]:
# Reading the data and fixing columns names

df = pd.read_csv(io.BytesIO(data_to_load['conn_attack.csv']), 
                 names=["record_id", "Duration_", "src_bytes", "dst_bytes"], 
                 index_col='record_id')
columns = df.columns

target = pd.read_csv(io.BytesIO(target_to_load['conn_attack_anomaly_labels.csv']), 
                 names=["id", "label"], 
                 index_col='id')

In [ ]:
df.head()

In [ ]:
target.head()

##### **Shape**

In [ ]:
df.shape

In [ ]:
columns.shape

In [ ]:
target.shape

##### **Analyzing the data frame** 

In [ ]:
df.info()

In [ ]:
df.describe()

##### **Data visualization**

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(18, 6))

# Data distributed on the "Duration_" feature only
sns.scatterplot(data=df.Duration_, ax=axes[0]).set(title="Duration_")

# Data distributed on the "src_bytes" feature only
sns.scatterplot(data=df.src_bytes, ax=axes[1]).set(title="src_bytes")

# Data distributed on the "dst_bytes" feature only
sns.scatterplot(data=df.dst_bytes, ax=axes[2]).set(title="dst_bytes")

## **Part 2 - Possible labels to compare**

---



---



##### **Assumption**

Essentially, we want to find ways to locate outliers without relying on any model to find the most appropriate model.


I found that the clustering algorithm will probably work because the data is mostly clustered in the center and only a small amount is scattered far away from it. As a result, isolation forests won't work, but DB SCAN will.

In [ ]:
q_low_1 = df.Duration_.quantile(0.01)
q_hi_1 = df.Duration_.quantile(0.99)

q_low_2 = df.src_bytes.quantile(0.01)
q_hi_2  = df.src_bytes.quantile(0.99)

q_low_3 = df.dst_bytes.quantile(0.01)
q_hi_3  = df.dst_bytes.quantile(0.99)

# Creating the guess
guess = np.zeros_like(df.src_bytes)

guess[(df.Duration_>q_hi_1) | (df.Duration_<q_low_1) | 
      (df.src_bytes>q_hi_2) | (df.src_bytes<q_low_2) | 
      (df.dst_bytes>q_hi_3) | (df.dst_bytes<q_low_3)] = 1 

## **Part 3 - Models**


---



---




In [ ]:
# Finding important features

df1 = df.drop(columns=columns[0])
df2 = df.drop(columns=columns[1])
df3 = df.drop(columns=columns[2])

df4 = df.drop(columns=[columns[0], columns[1]])
df5 = df.drop(columns=[columns[0], columns[2]])
df6 = df.drop(columns=[columns[1], columns[2]])

### ***3.1 - Isolation Forest***


In [ ]:
# Find if Isolation Forest might have good prediction

def run_isf_models(max_f_, df):
  isf_pred = []
  for i in range(100, 1000, 100):
      isf = IsolationForest(n_estimators=i, max_features=max_f_, n_jobs=-1)
      isf_labels = pd.Series(isf.fit_predict(df))
      isf_labels = isf_labels.map({1: 0, -1: 1})
      isf_pred.append([i, f1_score(y_true=guess, y_pred=isf_labels)])

  sns.lineplot(x=np.array(isf_pred)[:, 0], y=np.array(isf_pred)[:, 1])  

In [ ]:
# Running the model using all features

run_isf_models(3, df)

In [ ]:
# Running the model without "Duration_"

run_isf_models(2, df1)

In [ ]:
# Running the model without "src_bytes"

run_isf_models(2, df2)

In [ ]:
# Running the model without "dst_bytes"

run_isf_models(2, df3)

In [ ]:
# Running the model without "Duration_" , "src_bytes"

run_isf_models(1, df4)

In [ ]:
# Running the model without "Duration_" , "dst_bytes"

run_isf_models(1, df5)

In [ ]:
# Running the model without "src_bytes" , "dst_bytes"

run_isf_models(1, df6)

### ***3.2 - DBSCAN***

In [ ]:
# Mapping function to fit the correct labels

def _map(x):
  return 0 if x==-1 else 1
  
# Find if DBSCAN might have good prediction

def run_dbs_models(df):
  dbs_pred = []
  for i in range(50, 300, 50):
      dbs = DBSCAN(min_samples=i)
      dbs_labels = pd.Series(dbs.fit_predict(df))
      dbs_labels = dbs_labels.apply(lambda x: _map(x))
      dbs_pred.append([i, f1_score(y_true=guess, y_pred=dbs_labels)])

  sns.lineplot(x=np.array(dbs_pred)[:, 0], y=np.array(dbs_pred)[:, 1])

In [ ]:
# Running the model using all features

run_dbs_models(df)

In [ ]:
# Running the model without "Duration_"

run_dbs_models(df1)

In [ ]:
# Running the model without "src_bytes"

run_dbs_models(df2)

In [ ]:
# Running the model without "dst_bytes"

run_dbs_models(df3)

In [ ]:
# Running the model without "Duration_" , "src_bytes"

run_dbs_models(df4)

In [ ]:
# Running the model without "Duration_" , "dst_bytes"

run_dbs_models(df5)

In [ ]:
# Running the model without "src_bytes" , "dst_bytes"

# run_dbs_models(df6) --> Memory error!

## **Part 4 - Evaluation & Comparison**

---

---



In [ ]:
dbs = DBSCAN(min_samples=150, n_jobs=-1)
dbs_labels = pd.Series(dbs.fit_predict(df)).apply(lambda x: _map(x))

##### ***y_guess & y_dbs***

In [ ]:
confusion_matrix(y_true=guess, y_pred=dbs_labels)

In [ ]:
print(classification_report(y_true=guess, y_pred=dbs_labels))

##### ***y_guess & y_true***

In [ ]:
confusion_matrix(y_true=guess, y_pred=target.label)

In [ ]:
print(classification_report(y_true=guess, y_pred=target.label))

##### ***y_dbs & y_true***

In [ ]:
confusion_matrix(y_true=dbs_labels, y_pred=target.label)

In [ ]:
print(classification_report(y_true=dbs_labels, y_pred=target.label))

## **Part 5 - Export the output**

---

---

In [ ]:
DBS_labels = DBS_labels.set_axis(target.index)
df["is_anomaly"] = DBS_labels

output = df.drop(columns=["Duration_", "src_bytes", "dst_bytes"])

In [ ]:
output.info()

In [ ]:
output.head()

In [ ]:
output.describe()

In [ ]:
output.corr()

In [ ]:
output.to_csv('output.csv', encoding='utf-8', index=False)

## **Part 6 - Conclusion**

---

---



*   According to the F_scores, the isolation forest is not the right solution.

*   DBSCAN does fit this problem

*   While "src_bytes" is the most important feature, utilizing all features helps detect outliers more effectively.

*   Our guesses weren't accurate.

*   There is a high correlation between the real label and the prediction, which means that the prediction is very close to the real label. 



# **The End**

---
